In [110]:
import numpy as np
import math
import latexify

In [111]:
KERNEL_SIZE = 7
SIGMA = 15

In [112]:
@latexify.function(use_math_symbols=True)
def LoG(x, y, sigma):
    return ((- 1 / (np.pi * sigma**4) ) * (1 - (x**2 + y**2) / (2 * sigma**2) ) ) * np.exp(- (x**2 + y**2) / (2 * sigma**2))
    
def get_LoG_kernel(sigma : float, size : int) -> np.array:
    
    ker = np.empty((size, size), dtype=float)
    center_cell = size//2
    
    for ((x, y) , _) in np.ndenumerate(ker):
        ker[x , y] = LoG(x - center_cell , y - center_cell , sigma)
    
    return ker

LoG

In [113]:
ker = get_LoG_kernel(SIGMA, KERNEL_SIZE)
ker

array([[-5.79941977e-06, -5.93208997e-06, -6.01276548e-06,
        -6.03983782e-06, -6.01276548e-06, -5.93208997e-06,
        -5.79941977e-06],
       [-5.93208997e-06, -6.06700085e-06, -6.14903683e-06,
        -6.17656535e-06, -6.14903683e-06, -6.06700085e-06,
        -5.93208997e-06],
       [-6.01276548e-06, -6.14903683e-06, -6.23189882e-06,
        -6.25970432e-06, -6.23189882e-06, -6.14903683e-06,
        -6.01276548e-06],
       [-6.03983782e-06, -6.17656535e-06, -6.25970432e-06,
        -6.28760269e-06, -6.25970432e-06, -6.17656535e-06,
        -6.03983782e-06],
       [-6.01276548e-06, -6.14903683e-06, -6.23189882e-06,
        -6.25970432e-06, -6.23189882e-06, -6.14903683e-06,
        -6.01276548e-06],
       [-5.93208997e-06, -6.06700085e-06, -6.14903683e-06,
        -6.17656535e-06, -6.14903683e-06, -6.06700085e-06,
        -5.93208997e-06],
       [-5.79941977e-06, -5.93208997e-06, -6.01276548e-06,
        -6.03983782e-06, -6.01276548e-06, -5.93208997e-06,
        -5.7994197

In [114]:
import cv2
  
image = cv2.imread('OIP.jpeg')

image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  
cv2.imwrite("gray.jpeg", image)

True

In [115]:
original_rows, original_cols = image.shape
print(original_rows, original_cols)

padding = KERNEL_SIZE//2

top_left = np.zeros(shape=(padding, padding))
top_center = np.zeros(shape=(padding, original_cols))
top_right = np.zeros(shape=(padding, padding))
left = np.zeros(shape=(original_rows, padding))
right= np.zeros(shape=(original_rows, padding))
bottom_left = np.zeros(shape=(padding, padding))
bottom_center=np.zeros(shape=(padding, original_cols))
bottom_right=np.zeros(shape=(padding, padding))

padded_image =np.block([[top_left,      top_center,     top_right], 
                        [left,          image,          right],
                        [bottom_left,   bottom_center,  bottom_right]]
)

padded_image.shape

296 474


(302, 480)

In [116]:
convolved_image = np.zeros(shape=image.shape)

In [117]:
for i in range(convolved_image.shape[0]):
    for j in range(convolved_image.shape[1]):
        
        convolved_image[i , j] = np.sum(np.multiply(
            padded_image[i : i + padding*2 + 1 , j : j + padding*2 + 1 ],
            ker 
        ))

convolved_image = np.interp(
    convolved_image, 
    (convolved_image.min(), convolved_image.max()),
    (0, 255)
)

In [118]:
convolved_image

array([[234.36664655, 229.00503299, 223.8733162 , ..., 174.20581597,
        188.26900776, 202.9602617 ],
       [229.38550491, 222.605859  , 216.267812  , ..., 152.34112838,
        169.96605507, 188.14575207],
       [221.00113799, 213.31672773, 205.81095314, ..., 131.94699165,
        152.68205436, 173.74974626],
       ...,
       [253.92843146, 253.45331381, 253.00898716, ..., 148.12754127,
        165.96261004, 183.92251747],
       [254.3369975 , 254.12682858, 253.79601912, ..., 156.30248684,
        172.77567766, 189.45974785],
       [254.79374797, 254.69731367, 254.43616994, ..., 176.88370762,
        189.64638296, 202.60532293]])

In [119]:
cv2.imwrite("out.jpeg", convolved_image)

True

In [120]:
radius = int (np.sqrt(2) * SIGMA)
print(radius)

rows, columns = image.shape
print(image.shape)

local_maxima = []

for i in range(radius, rows - radius): # check 
    for j in range(radius, columns - radius):
        if image[i, j] == np.amax(image[i - radius : i + radius, j - radius : j + radius ] ):
            local_maxima.append( (i,j))

print(len(local_maxima))


21
(296, 474)
44


In [121]:
rgb_image = cv2.imread("OIP.jpeg")

for i, j in local_maxima: 
    cv2.circle(rgb_image, (i, j), radius, (255, 0, 0), thickness=1, lineType=2, shift=0)

cv2.imwrite("circled.jpeg", rgb_image)


True